# Segmenting and Clustering Neighborhoods in Toronto -- 1

### Use wikipedia to scrap for data to create dataframe

The dataframe will contain three columns: 

    - PostalCode
    - Borough
    - Neighborhood

1) Only process the cells that have an assigned borough. Ignore cells with a borough that is **Not assigned**.

2) If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

3) More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.



- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

#### **Submit link when finished**

---

#### Import libraries

In [1]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
import pandas as pd
import numpy as np

Use the `requests` library to download the webpage. Save the text of the response as a variable named `html_data`.

In [2]:
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=946126446"

html_data  = requests.get(url).text 

Parse the html data using `beautiful_soup`.

In [3]:
soup = BeautifulSoup(html_data,"html5lib")  # create a soup object using the variable 'html_data'

Check title to ensure correct webpage

In [4]:
print(soup.title)

<title>List of postal codes of Canada: M - Wikipedia</title>


Using beautiful soup extract the table and store it into a dataframe. The dataframe should have columns **PostalCode**, **Borough**, and **Neighborhood**. Fill in each variable with the correct data from the list `col`. 

Hint: Print the `col` list to see what data to use


In [5]:
toronto_data = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])

# print(soup.find("table",{"class":"wikitable sortable"}).find("tbody").find_all("tr"))

for row in soup.find("table",{"class":"wikitable sortable"}).find("tbody").find_all("tr"):
    col = row.find_all("td")
    if(col):
        postalCode =col[0].text.strip()
        borough = col[1].text.strip()
        neighborhood = col[2].text.strip()
        toronto_data = toronto_data.append({"PostalCode":postalCode, "Borough":borough, "Neighborhood":neighborhood}, ignore_index=True)


Check the dataframe for quick summary

In [6]:
toronto_data.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [7]:
toronto_data.describe()

,PostalCode,Borough,Neighborhood
count,287,287,287
unique,180,11,209
top,M8Y,Not assigned,Not assigned
freq,8,77,77


In [8]:
toronto_data.shape

(287, 3)

Dupicate dataframe so we can save the OG df as reference

In [9]:
toronto_data_new = toronto_data

---

### 1) Ignore cells with a borough that is **Not assigned**.

In [10]:
toronto_data_new["Borough"] = toronto_data_new["Borough"].replace("Not assigned", np.nan)

toronto_data_new.head()

,PostalCode,Borough,Neighborhood
0,M1A,NaN,Not assigned
1,M2A,NaN,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Lets count the missing values in each column

In [11]:
missing_data = toronto_data_new.isnull()
missing_data.head()

,PostalCode,Borough,Neighborhood
0,False,True,False
1,False,True,False
2,False,False,False
3,False,False,False
4,False,False,False


In [12]:
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")  

PostalCode
False    287
Name: PostalCode, dtype: int64

Borough
False    210
True      77
Name: Borough, dtype: int64

Neighborhood
False    287
Name: Neighborhood, dtype: int64



"True" represents a missing value, "False"  means the value is present in the dataset.
We can see **Borough** has **77 missing value**.
Lets drop these 77 rows.

In [13]:
toronto_data_new = toronto_data_new.dropna(subset=["Borough"], axis=0)

# Lets reset index, because we dropped rows
toronto_data_new = toronto_data_new.reset_index(drop=True)

toronto_data_new.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


---

### 2) If a cell has a **borough** but a **Not assigned** neighborhood, then the **neighborhood** will be the same as the **borough**.

In [14]:
toronto_data_new['Neighborhood'] == 'Not assigned'

0      False
1      False
2      False
3      False
4      False
       ...  
205    False
206    False
207    False
208    False
209    False
Name: Neighborhood, Length: 210, dtype: bool

In [15]:
missing_data = toronto_data_new.isnull()
missing_data.head()

,PostalCode,Borough,Neighborhood
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False


In [16]:
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")  

PostalCode
False    210
Name: PostalCode, dtype: int64

Borough
False    210
Name: Borough, dtype: int64

Neighborhood
False    210
Name: Neighborhood, dtype: int64



No `Not assigned` values found in column Neighborhood

---

### 3) Check if more than one neighborhood can exist in one postal code area.

In [17]:
toronto_data_new["PostalCode"].value_counts()

M9V    8
M8Y    8
M5V    7
M8Z    5
M4V    5
      ..
M3A    1
M5E    1
M4R    1
M4J    1
M4H    1
Name: PostalCode, Length: 103, dtype: int64

If PositalCode is the same, then group them up

In [18]:
toronto_data_new = toronto_data_new.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(list)
toronto_data_new = toronto_data_new.sample(frac=1).reset_index()
toronto_data_new['Neighborhood'] = toronto_data_new['Neighborhood'].str.join(', ')
toronto_data_new

,PostalCode,Borough,Neighborhood
0,M2M,North York,"Newtonbrook, Willowdale"
1,M1N,Scarborough,"Birch Cliff, Cliffside West"
2,M5C,Downtown Toronto,St. James Town
3,M8W,Etobicoke,"Alderwood, Long Branch"
4,M4T,Central Toronto,"Moore Park, Summerhill East"
...,...,...,...
98,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade
99,M5N,Central Toronto,Roselawn
100,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter"
101,M6L,North York,"Downsview, North Park, Upwood Park"


---

### Final dataframe after data cleaning

In [19]:
toronto_data_new.shape

(103, 3)